In [1]:
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from correlations import correlations
import numpy as np

# Reproduce equal voids

In [3]:
corr = correlations.Correlator()

def eig2mat(e1, e2, theta):
    '''Return the element of the matrix with eigen values e1, e2 in (x, y).'''
    cos = np.cos(theta)
    sin = np.sin(theta)
    return [
        e1 * cos**2 + e2 * sin**2,  # xx
        e1 * sin**2 + e2 * cos**2,  # yy
        np.inf,                      # zz
        (e2-e1) * cos * sin,         # xy
        np.inf,                      # xz
        np.inf,                      # yz
    ]
kappa = 1

corr.add_point([0, 0, 0], ['density', 'density_gradient', 'hessian'], 1, 
               constrains={'density': [3], 'density_gradient': [0, 0, np.inf], 
                           'hessian': [-kappa, -kappa, np.inf,
                                       0,      np.inf, np.inf]})

r = 5
for phi in [0, 2*np.pi/3, -2*np.pi/3]:
    x = r * np.cos(phi)
    y = r * np.sin(phi)
    corr.add_point([x, y, 0], ['density', 'density_gradient', 'hessian'], 1,
                   constrains={'density': [-4], 'density_gradient': [0, 0, np.inf], 
                               'hessian': eig2mat(kappa, 2*kappa, phi+np.pi/2)})

grid = np.linspace(-r, r, 11)
xy = []
for x, y in itertools.product(grid, grid):
    corr.add_point([x, y, 0], ['density'], 1)
    xy.append((x, y))
    

In [ ]:
corr.cov_c

  0%|          | 0/13041 [00:00<?, ?it/s]

Running with 4 processes.


100%|██████████| 13041/13041 [02:33<00:00, 85.03it/s]

In [ ]:
from scipy.interpolate import interp2d

# o stands for original, h for high resolution
ox, oy = np.meshgrid(grid, grid)
ox, oy = ox.flatten(), oy.flatten()
odata = corr.mean_c

hx, hy = np.meshgrid(np.linspace(-r, r, 100), np.linspace(-r, r, 100))
interp = np.vectorize(interp2d(ox, oy, odata, kind='quintic'))
hdata = interp(hx, hy)

# plt.pcolormesh(hx, hy, hdata, cmap='RdYlBu')
plt.contourf(hx, hy, hdata.T, cmap='RdYlBu_r', vmin=-4, vmax=4)
plt.gca().set_aspect('equal')